<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/10.%20Chapter10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 10: Introduction to Artificial Neural Networks with Keras

Bab ini memperkenalkan konsep dasar di balik **Artificial Neural Networks (ANN)**, mulai dari inspirasi biologisnya hingga implementasi praktis menggunakan API tingkat tinggi, **Keras**.

* **Dari Neuron Biologis ke Neuron Buatan**: Bab ini dimulai dengan sejarah singkat ANN, terinspirasi dari cara kerja otak. Diperkenalkan arsitektur paling awal seperti **Perceptron**, yang merupakan sebuah unit komputasi sederhana (*Threshold Logic Unit* atau TLU) yang dapat melakukan klasifikasi biner linier. Namun, Perceptron memiliki keterbatasan signifikan (misalnya, tidak bisa menyelesaikan masalah XOR), yang menyebabkan periode stagnasi dalam riset ANN ("AI winter").

* **Multilayer Perceptron (MLP) dan Backpropagation**:
    * **MLP** adalah solusi untuk keterbatasan Perceptron, yang terdiri dari satu *input layer*, satu atau lebih *hidden layers*, dan satu *output layer*. Dengan lapisan yang cukup, MLP dapat memecahkan masalah yang sangat kompleks.
    * **Backpropagation** adalah algoritma training fundamental untuk MLP. Ia bekerja dengan cara:
        1.  Melakukan *forward pass* untuk menghitung output dan *loss*.
        2.  Melakukan *reverse pass* untuk mengukur kontribusi error dari setiap koneksi.
        3.  Menggunakan gradien yang dihitung untuk memperbarui bobot model melalui *Gradient Descent*.
    * **Fungsi Aktivasi**: Diperkenalkan fungsi aktivasi non-linier seperti **ReLU**, **tanh**, dan **sigmoid**, yang sangat penting karena tanpa non-linearitas, menumpuk beberapa lapisan sama saja dengan satu lapisan linier.

* **Implementasi MLP dengan Keras**:
    * **Keras** adalah API tingkat tinggi yang intuitif untuk membangun, melatih, dan mengevaluasi jaringan saraf. `tf.keras` adalah implementasi Keras yang terintegrasi penuh dengan TensorFlow.
    * **Sequential API**: Cara paling sederhana untuk membangun model, yaitu dengan membuat tumpukan lapisan (*stack of layers*) secara berurutan.
    * **Functional API**: Memungkinkan pembangunan arsitektur yang lebih kompleks, seperti jaringan dengan beberapa input atau output (contohnya, arsitektur *Wide & Deep*).
    * **Subclassing API**: Memberikan fleksibilitas maksimum dengan memungkinkan kita membuat model sebagai *class* Python, berguna untuk model yang sangat dinamis.

* **Praktik Terbaik**:
    * **Menyimpan dan Memuat Model**: Keras memudahkan penyimpanan arsitektur model, bobot, dan status optimizer.
    * **Callbacks**: Fungsi yang dapat dieksekusi pada berbagai titik selama training, seperti `ModelCheckpoint` untuk menyimpan model terbaik dan `EarlyStopping` untuk menghentikan training lebih awal.
    * **TensorBoard**: Alat visualisasi untuk memantau metrik training, melihat arsitektur model, dan lainnya.
    * ***Hyperparameter Tuning***: Teknik untuk menemukan kombinasi *hyperparameter* terbaik (misalnya, jumlah lapisan, jumlah neuron, *learning rate*) menggunakan *wrapper* Scikit-Learn dan alat seperti `RandomizedSearchCV`.

### 1. Membangun Pengklasifikasi Gambar dengan Sequential API
Kita akan menggunakan dataset Fashion MNIST, versi yang lebih menantang dari MNIST.

```python
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Memuat dataset Fashion MNIST
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Membagi data training menjadi training dan validation set
# dan melakukan normalisasi piksel (scaling) ke rentang 0-1
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

# Membangun model menggunakan Sequential API
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # Meratakan input 28x28 menjadi vektor 1D
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax") # Output layer dengan 10 neuron (1 per kelas)
])

# Menampilkan ringkasan model
model.summary()
```

### 2. Mengompilasi dan Melatih Model
Setelah model dibuat, kita perlu mengompilasinya dengan menentukan *loss function*, *optimizer*, dan metrik, lalu melatihnya.

```python
# Mengompilasi model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

# Melatih model
# history akan menyimpan metrik training dan validasi di setiap epoch
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

# Plot learning curves
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # Mengatur batas vertikal [0, 1]
plt.show()
```

### 3. Evaluasi dan Prediksi
Setelah model dilatih, kita bisa mengevaluasinya pada *test set* dan menggunakannya untuk prediksi.

```python
# Evaluasi pada test set
print("Hasil evaluasi pada test set:")
model.evaluate(X_test, y_test)

# Membuat prediksi pada beberapa instance baru
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_pred = np.argmax(y_proba, axis=-1)

print("\nPrediksi probabilitas:\n", y_proba.round(2))
print("Prediksi kelas:", np.array(class_names)[y_pred])
```

### 4. Membangun Model Regresi dengan Functional API
Functional API lebih fleksibel dan berguna untuk arsitektur yang lebih kompleks, seperti model *Wide & Deep*.

```python
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Memuat dan mempersiapkan dataset California Housing
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Membangun model Wide & Deep dengan Functional API
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_, hidden2]) # Skip connection (jalur 'wide')
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

# Mengompilasi dan melatih model
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
print("\nMSE pada test set:", mse_test)
```
Contoh di atas menunjukkan cara membuat model dengan koneksi non-sekuensial (*skip connection*), yang merupakan keunggulan utama dari Functional API.

